In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import matplotlib.patches as patches
import matplotlib.patheffects as PathEffects
from scipy.stats import mannwhitneyu, ks_2samp
# Global plotting style
plt.rcParams["font.size"] = 13
plt.rcParams["axes.linewidth"] = 1.3
plt.rcParams["xtick.direction"] = "in"
plt.rcParams["ytick.direction"] = "in"
plt.rcParams["xtick.top"] = True
plt.rcParams["ytick.right"] = True
plt.rcParams["figure.dpi"] = 70

## User configuration

In [ ]:
# ===================================================================
# USER CONFIGURATION
# ===================================================================
# This is the configuration cell. The user is expected to edit:
#
#   1) Catalogue selection
#      - Used for the P–R approximate landscape 
#        and the two–panel P–R + P–ρ density plot.
#        Not used by the DR25 "exact boundaries" plot.
#
#   2–4) Plot settings required for ALL plots
#        (approximate P–R, density P–R, density P–ρ, exact/DR25).
#
#   5–6) Settings affecting only the P–ρ/P–R density plot and its
#        Monte Carlo statistics.
#
#   7) Optional tag appended to all output filenames.
#
# ===================================================================

# -------------------------------------------------------------------
# 1) Catalogue used for the landscape and two–panel plots
#    Affects: P–R approximate landscape, P–R and P–ρ density plot
#    (NOT the DR25 "exact boundaries" plot, which uses its own file)
# -------------------------------------------------------------------
# Options:
#   - "NEA"          : NASA Exoplanet Archive
#   - "Exoplanet.eu" : exoplanet.eu catalogue
catalog = "NEA"


# -------------------------------------------------------------------
# 2) Neptunian landscape boundaries and labels
#    Affects: all P–R panels (approximate, density left, exact/DR25)
# -------------------------------------------------------------------
# Boundaries to draw in the P–R plots (comma–separated list):
#   - "desert" -> Castro-González et al. (2024) Neptunian desert
#   - "ridge"  -> Neptunian ridge
show_boundaries = "desert,ridge"

# Region labels to display (comma–separated subset of):
#   "desert", "ridge", "savanna"
show_labels = "desert,ridge,savanna"

# Whether to overplot the classical Neptunian desert limits from
# Mazeh et al. (2016) as dotted lines in all P–R panels.
show_mazeh_boundaries = False


# -------------------------------------------------------------------
# 3) User planets (used by ALL plots)
#    Affects: P–R approximate, density P–R panel, exact/DR25, P–ρ panel
# -------------------------------------------------------------------
# Expected fields for each planet dictionary:
#   name             : str, planet name
#   P_days           : float, orbital period [days]
#   R_Rearth         : float, radius [R⊕]
#   R_err_up/down    : float, 1σ radius errors [R⊕]
#   M_Mearth         : float, optional mass [M⊕]
#   M_err_up/down    : float, 1σ mass errors [M⊕]
#   rho_cgs          : float, optional density [g cm⁻3]
#   rho_err_up/down  : float, 1σ density errors [g cm⁻3]
#   M_upper_3sigma   : bool, if True, interpret mass as 3σ upper limit
#   rho_upper_3sigma : bool, if True, interpret density as 3σ upper limit
#   color            : marker color in all plots
#   dx_label,dy_label: label offsets in P–R plots (linear units)
#
# Notes:
# - If M and R are provided, ρ is computed with error propagation.
# - If rho_cgs is provided, it is used directly in the density panel.
# - For the density trend:
#   * If neither (M,R) nor rho_cgs are available, the planet is skipped.
#   * Upper limits are drawn with downward arrows in the P–ρ panel.

user_planets = [


    # TOI-849 b
    # P = 0.7655 d, Rp = 3.44(+0.16/-0.11) R⊕
    dict(
        name = "TOI-849 b",
        P_days = 0.7655,
        R_Rearth = 3.44,
        R_err_up = 0.16,
        R_err_down = 0.11,
        color = "#FF99E6",
        dx_label = 0.0,
        dy_label = 0.0,
    ),
    
    # TOI-5005 b
    # P = 6.3 d, Rp = 6.25 ± 0.24 R⊕, Mp = 32.7 ± 5.9 M⊕
    dict(
        name = "TOI-5005 b",
        P_days = 6.3,
        R_Rearth = 6.25,
        R_err_up = 0.24,
        R_err_down = 0.24,
        M_Mearth = 32.7,
        M_err_up = 5.9,
        M_err_down = 5.9,
        color = "#80BFFF",
        dx_label = 0.0,
        dy_label = 0.0,
    ),

]

# To disable user planets entirely:
# user_planets = []


# -------------------------------------------------------------------
# 4) Display mode for user planets
# -------------------------------------------------------------------
# Options:
#   - "box"    -> inline labels inside the plot (colored text boxes)
#   - "legend" -> classic legend with one symbol per planet
legend_or_box = "legend"


# -------------------------------------------------------------------
# 5) Radius range and density–plot configuration
# -------------------------------------------------------------------
# Radius interval [R⊕] used to define the Neptunian range and to select
# planets for the density trend.
R_band_min = 4.5
R_band_max = 7.5

# P–ρ panel configuration:
#   - density_xlim_max: also sets the upper-period cut for the savanna
#     sample (P ≤ density_xlim_max) in the statistics.
#   - density_ymax: vertical axis limit (display only).
density_ymax     = 3.2    # [g cm⁻3]
density_xlim_max = 23.0   # [days]


# -------------------------------------------------------------------
# 6) Precision cuts for the density trend (P–ρ panel only)
# -------------------------------------------------------------------
# Planets included in the P–ρ density trend must satisfy one of:
#
# (A) If max_rel_err_rho_pct > 0:
#       Use a density-based criterion:
#           sigma_rho / rho  <  max_rel_err_rho_pct / 100
#
# (B) If max_rel_err_rho_pct <= 0:
#       Use radius- and mass-based criteria:
#           sigma_R / R  <  max_frac_err_R_pct / 100
#           sigma_M / M  <  max_frac_err_M_pct / 100
#
# These cuts affect only the P–ρ panel and its Monte Carlo statistics.
max_rel_err_rho_pct = 33.0  # %
#max_frac_err_R_pct  = 33.0  # %
#max_frac_err_M_pct  = 33.0  # %

# Whether to include user planets in the density Monte Carlo statistics
include_user_planets_in_density_stats = False


# -------------------------------------------------------------------
# 7) Optional tag for output filenames
# -------------------------------------------------------------------
# If empty, figures are saved with default names, e.g.:
#   nep_lands.pdf
# If non-empty (e.g. "TOI-5005"), file names become:
#   <base_name>_<plot_tag>.pdf/png
plot_tag = ""  # e.g. "TOI-5005"

## Main code block (collapsed)
#### Run these five cells before running the plotting cells

In [ ]:
# ---------------------------------------------------------
# Global flags (used by all plots)
# ---------------------------------------------------------
labels_list = [s.strip() for s in show_labels.split(",") if s.strip()]
show_label_desert  = "desert"  in labels_list
show_label_ridge   = "ridge"   in labels_list
show_label_savanna = "savanna" in labels_list

boundaries_list = [s.strip() for s in show_boundaries.split(",") if s.strip()]
show_desert_boundaries = "desert" in boundaries_list
show_ridge_boundaries  = "ridge"  in boundaries_list

# -------------------------------------------------------------------
# Paths (expect 'aux/' and 'catalogue_data/' next to this file)
# -------------------------------------------------------------------
base_dir = os.getcwd()
aux_dir = os.path.join(base_dir, "aux")
catalogue_data_dir = os.path.join(base_dir, "cat")
output_dir = os.path.join(base_dir, "output")
os.makedirs(output_dir, exist_ok=True)


def make_output_name(basename: str) -> str:
    """
    Optionally append the global 'plot_tag' to a base filename.

    If 'plot_tag' is undefined or empty, returns 'basename' unchanged.
    Otherwise returns '<root>_<plot_tag><ext>'.
    """
    tag = globals().get("plot_tag", "")
    if not tag:
        return basename
    root, ext = os.path.splitext(basename)
    return f"{root}_{tag}{ext}"


# -------------------------------------------------------------------
# Plot colours and styles (fixed defaults)
# -------------------------------------------------------------------
color_text       = "white"   # DESERT / RIDGE / SAVANNA labels
color_boundaries = "black"
style_boundaries = "dashed"
color_map        = "magma_r"

# P–R shaded regions (desert+ridge vs savanna)
color_ridge_desert = "#CC79A7"
color_savanna      = "#56B4E9"
alpha_band         = 0.65

# User-planet marker sizes (all plots)
user_marker_size_PR      = 190  # P–R panels
user_marker_size_rho     = 13   # P–ρ normal detections
user_marker_size_rho_UL  = 13   # P–ρ upper limits
user_marker_size_legend  = 13   # Legend symbols for user planets

# -------------------------------------------------------------------
# Physical constants
# -------------------------------------------------------------------
rho_earth = 5.51  # g cm^-3


def compute_density_from_MR(M_Mearth, R_Rearth, dM_up, dM_down, dR_up, dR_down):
    """
    Compute bulk density in Earth units (ρ / ρ⊕) and its 1σ uncertainty
    from mass and radius (M in M⊕, R in R⊕), assuming independent errors.
    """
    M = float(M_Mearth)
    R = float(R_Rearth)

    sigma_M = 0.5 * (abs(dM_up) + abs(dM_down))
    sigma_R = 0.5 * (abs(dR_up) + abs(dR_down))

    if M <= 0 or R <= 0:
        return np.nan, np.nan

    rho_re = M / (R**3)
    frac_M = sigma_M / M if M > 0 else 0.0
    frac_R = sigma_R / R if R > 0 else 0.0

    sigma_rho_re = rho_re * np.sqrt(frac_M**2 + (3.0 * frac_R)**2)
    return rho_re, sigma_rho_re


def get_user_density_arrays():
    """
    Build arrays of P, rho, sigma_rho, etc. for planets defined in
    the global `user_planets` list.

    Conventions:
    - If the dictionary contains 'rho_cgs', it is interpreted as the
      physical density in g cm^-3.
    - If it contains both M_Mearth and R_Rearth, the density is
      computed from M and R (also in g cm^-3).
    """

    P_list         = []
    rho_list       = []
    sigma_rho_list = []
    is_upper_list  = []
    labels         = []
    colors         = []
    dxs            = []
    dys            = []
    missing_names  = []

    for pl in user_planets:
        name = pl.get("name", "user-planet")
        col  = pl.get("color", "black")
        dx   = float(pl.get("dx_label", 0.0))
        dy   = float(pl.get("dy_label", 0.0))

        # Orbital period
        P = pl.get("P_days", None)
        if P is None or (not np.isfinite(P)):
            missing_names.append(name)
            continue

        # Radius (for Neptunian-band selection)
        R = pl.get("R_Rearth", np.nan)
        if np.isfinite(R):
            if (R < R_band_min) or (R > R_band_max):
                # skip outside Neptunian band
                continue

        # Decide whether to use rho_cgs or (M, R)
        has_rho = ("rho_cgs" in pl) and (pl["rho_cgs"] is not None)
        has_MR  = (
            ("M_Mearth" in pl) and (pl["M_Mearth"] is not None)
            and ("R_Rearth" in pl) and (pl["R_Rearth"] is not None)
        )

        if not has_rho and not has_MR:
            missing_names.append(name)
            continue

        if has_rho:
            # Direct density [g cm^-3]
            rho = float(pl["rho_cgs"])
            d_up = float(pl.get("rho_err_up", 0.0))
            d_dn = float(pl.get("rho_err_down", 0.0))
            sigma_rho = 0.5 * (abs(d_up) + abs(d_dn))

            if not np.isfinite(rho):
                missing_names.append(name)
                continue

        else:
            # Compute density from M and R in Earth units
            M     = float(pl["M_Mearth"])
            R     = float(pl["R_Rearth"])
            dM_up = float(pl.get("M_err_up", 0.0))
            dM_dn = float(pl.get("M_err_down", 0.0))
            dR_up = float(pl.get("R_err_up", 0.0))
            dR_dn = float(pl.get("R_err_down", 0.0))

            rho_re, sigma_rho_re = compute_density_from_MR(
                M, R, dM_up, dM_dn, dR_up, dR_dn,
            )

            if (not np.isfinite(rho_re)) or (rho_re <= 0.0):
                missing_names.append(name)
                continue

            # Convert to g cm^-3
            rho       = rho_re * rho_earth
            sigma_rho = sigma_rho_re * rho_earth

        is_upper = bool(
            pl.get("M_upper_3sigma", False) or pl.get("rho_upper_3sigma", False)
        )

        P_list.append(P)
        rho_list.append(rho)
        sigma_rho_list.append(sigma_rho)
        is_upper_list.append(is_upper)
        labels.append(name)
        colors.append(col)
        dxs.append(dx)
        dys.append(dy)

    return (
        np.array(P_list),
        np.array(rho_list),
        np.array(sigma_rho_list),
        np.array(is_upper_list, dtype=bool),
        labels,
        colors,
        dxs,
        dys,
        missing_names,
    )


def outlined_text_effect(linewidth=2.2):
    """Return a black outline effect for text artists."""
    return [PathEffects.withStroke(linewidth=linewidth, foreground="black")]


def load_catalog_PR(catalog, precision_radius=30.0, precision_mass=0.0):
    """
    Load a period–radius catalogue and apply basic quality cuts.

    Parameters
    ----------
    catalog : {"NEA", "Exoplanet.eu"}
        Source catalogue to load.
    precision_radius : float, optional
        Maximum allowed fractional 1σ uncertainty in radius (percent).
        If <= 0, no radius precision cut is applied.
    precision_mass : float, optional
        Maximum allowed fractional 1σ uncertainty in mass (percent).
        If <= 0, no mass precision cut is applied.

    Returns
    -------
    df : pandas.DataFrame
        Filtered catalogue (P > 0, R > 0, and passing any precision cuts).
    P_catalog : np.ndarray
        Orbital periods in days.
    R_catalog : np.ndarray
        Planet radii in Earth radii (R⊕).
    """
    if catalog == "NEA":
        nea_dir = os.path.join(catalogue_data_dir, "NEA")
        files = sorted(
            f for f in os.listdir(nea_dir) if f.lower().endswith(".csv")
        )
        if not files:
            raise FileNotFoundError(f"No CSV files found in {nea_dir}.")
        df = pd.read_csv(os.path.join(nea_dir, files[-1]), comment="#")

    elif catalog == "Exoplanet.eu":
        exo_dir = os.path.join(catalogue_data_dir, "Exoplanet.eu")
        files = sorted(
            f for f in os.listdir(exo_dir) if f.lower().endswith(".csv")
        )
        if not files:
            raise FileNotFoundError(f"No CSV files found in {exo_dir}.")

        df = pd.read_csv(os.path.join(exo_dir, files[-1]))

        # Standardise column names to the NEA convention
        rename_map = {
            "orbital_period":   "pl_orbper",
            "radius":           "pl_rade",
            "radius_error_min": "pl_radeerr2",
            "radius_error_max": "pl_radeerr1",
        }
        if "mass" in df.columns:
            rename_map["mass"] = "pl_bmasse"
        if "mass_error_min" in df.columns:
            rename_map["mass_error_min"] = "pl_bmasseerr2"
        if "mass_error_max" in df.columns:
            rename_map["mass_error_max"] = "pl_bmasseerr1"

        df = df.rename(columns=rename_map)

        # Unit conversions for exoplanet.eu → Earth units

        # Mass: M_Jup → M_earth
        if "pl_bmasse" in df.columns:
            MJUP_TO_MEARTH = 317.828
            df["pl_bmasse"] *= MJUP_TO_MEARTH
            if "pl_bmasseerr1" in df.columns:
                df["pl_bmasseerr1"] *= MJUP_TO_MEARTH
            if "pl_bmasseerr2" in df.columns:
                df["pl_bmasseerr2"] *= MJUP_TO_MEARTH

        # Radius: R_Jup → R_earth
        if "pl_rade" in df.columns:
            RJUP_TO_REARTH = 11.209
            df["pl_rade"] *= RJUP_TO_REARTH
            if "pl_radeerr1" in df.columns:
                df["pl_radeerr1"] *= RJUP_TO_REARTH
            if "pl_radeerr2" in df.columns:
                df["pl_radeerr2"] *= RJUP_TO_REARTH

    else:
        raise ValueError("catalog must be 'NEA' or 'Exoplanet.eu'.")

    # ----------------------------------------------------------------
    # Precision cuts
    # ----------------------------------------------------------------
    if precision_mass > 0.0 and "pl_bmasse" in df.columns:
        M_pl  = df["pl_bmasse"].values
        M_err = (np.abs(df["pl_bmasseerr1"].values) +
                 np.abs(df["pl_bmasseerr2"].values)) / 2.0
        mask_M = (M_err / M_pl) < (precision_mass / 100.0)
        df = df.loc[mask_M]

    if precision_radius > 0.0 and {"pl_radeerr1", "pl_radeerr2"}.issubset(df.columns):
        R_pl  = df["pl_rade"].values
        R_err = (np.abs(df["pl_radeerr1"].values) +
                 np.abs(df["pl_radeerr2"].values)) / 2.0
        mask_R = (R_err / R_pl) < (precision_radius / 100.0)
        df = df.loc[mask_R]

    # Extract P and R and remove non–physical entries
    P_catalog = df["pl_orbper"].values
    R_catalog = df["pl_rade"].values

    mask_phys = (P_catalog > 0.0) & (R_catalog > 0.0)
    P_catalog = P_catalog[mask_phys]
    R_catalog = R_catalog[mask_phys]
    df = df.loc[mask_phys]

    return df.reset_index(drop=True), P_catalog, R_catalog


df_cat, P_catalog, R_catalog = load_catalog_PR(catalog)


def load_frozen_kde_points(catalog):
    """Load pre–computed KDE samples for the selected catalogue."""
    fname = os.path.join(aux_dir, f"kde_points_{catalog}.npz")

    if not os.path.exists(fname):
        raise FileNotFoundError(
            f"KDE file '{os.path.basename(fname)}' not found in directory: {aux_dir}"
        )

    data = np.load(fname)
    return data["logP_kde"], data["logR_kde"]


# Load KDE points for the selected catalogue
logP_kde, logR_kde = load_frozen_kde_points(catalog)
# len(logP_kde), len(logR_kde)

def setup_PR_axes(ax):
    """
    Configure the shared P–R axes in log10 space (used in all P–R panels).

    x-axis: log10(P / days), ticks labelled in days.
    y-axis: log10(R / R⊕),    ticks labelled in Earth radii.
    """
    ax.set_xticks(
        [-0.4, 0, 0.4, 0.8, 1.2, 1.6, 2.0, 2.4],
        [
            round(10**-0.4, 1),
            "1.0",
            round(10**0.4, 1),
            round(10**0.8, 1),
            round(10**1.2, 1),
            round(10**1.6, 1),
            round(10**2.0, 1),
            round(10**2.4, 1),
        ],
        fontsize=13.5,
    )

    ax.set_yticks(
        [0, 0.2, 0.4, 0.6, 0.8, 1.0, 1.2, 1.4],
        [
            "1.0",
            round(10**0.2, 1),
            round(10**0.4, 1),
            round(10**0.6, 1),
            round(10**0.8, 1),
            round(10**1.0, 1),
            round(10**1.2, 1),
            round(10**1.4, 1),
        ],
        fontsize=13.5,
    )

    ax.set_xlim(-0.5, 2.2)
    ax.set_ylim(0.07, 1.36)

    ax.set_xlabel("Orbital period (days)", fontsize=16)
    ax.set_ylabel(r"Planet radius ($\rm R_{\oplus}$)", fontsize=16)

    ax.tick_params(
        which="both",
        direction="in",
        length=6,
        width=1.3,
        color="k",
        top=True,
        right=True,
        bottom=True,
        left=True,
    )

    # Make sure ticks and labels are drawn on top of everything else
    ax.set_axisbelow(False)
    for spine in ax.spines.values():
        spine.set_zorder(10000)

    for axis_obj in [ax.xaxis, ax.yaxis]:
        for tick in axis_obj.get_major_ticks() + axis_obj.get_minor_ticks():
            tick.tick1line.set_zorder(20000)
            tick.tick2line.set_zorder(20000)
            tick.label1.set_zorder(20000)
            tick.label2.set_zorder(20000)
            tick.tick1line.set_clip_on(False)
            tick.tick2line.set_clip_on(False)


def plot_user_planets_PR(ax, legend_or_box):
    """
    Plot user planets on a P–R diagram in log10 space.

    - Uses the global `user_planets` list.
    - In 'legend' mode, only user planets appear in the legend.
    - In 'box' mode, labels are drawn as inline text boxes.
    """
    if not user_planets:
        return

    handle_list = []

    for pl in user_planets:
        try:
            P  = pl["P_days"]
            R  = pl["R_Rearth"]
            Ru = pl.get("R_err_up", 0.0)
            Rd = pl.get("R_err_down", 0.0)
            # kept for possible future x-error bars:
            Pu = pl.get("P_err_up", 0.0)
            Pd = pl.get("P_err_down", 0.0)

            col  = pl.get("color", "gold")
            name = pl.get("name", "planet")
            dx   = pl.get("dx_label", 0.0)
            dy   = pl.get("dy_label", 0.0)

            if P <= 0 or R <= 0:
                continue

            logP = np.log10(P)
            logR = np.log10(R)

            sc = ax.scatter(
                logP,
                logR,
                fc=col,
                ec="k",
                s=user_marker_size_PR,
                zorder=200000,
                marker="h",
                label=name if legend_or_box == "legend" else None,
            )

            # Vertical error bars in log(R) from asymmetric linear errors
            yerr_up = 0.0
            yerr_dn = 0.0
            if Ru > 0:
                yerr_up = np.log10(R + Ru) - logR
            if Rd > 0 and R - Rd > 0:
                yerr_dn = logR - np.log10(R - Rd)

            if yerr_up > 0 or yerr_dn > 0:
                ax.errorbar(
                    logP,
                    logR,
                    yerr=[[yerr_dn], [yerr_up]],
                    c="k",
                    lw=2,
                    zorder=199999,
                    capsize=3,
                )

            if legend_or_box == "box":
                ax.text(
                    np.log10(P + dx) if dx != 0 else logP,
                    np.log10(R + dy) if dy != 0 else logR,
                    name,
                    fontsize=10.5,
                    va="bottom",
                    ha="center",
                    bbox=dict(
                        boxstyle="round",
                        facecolor=col,
                        alpha=0.85,
                        pad=0.4,
                    ),
                    c="white",
                    weight="bold",
                    zorder=10000,
                )
            else:
                handle_list.append(sc)

        except Exception:
            continue

    if legend_or_box == "legend":
        # Return the list of artist handles so that the calling
        # function can build a dedicated legend (e.g. combined
        # with boundary legends in different locations).
        return handle_list

    # In 'box' mode or if no valid planets, no legend handles are used.
    return []


def plot_user_planets_density(ax, legend_or_box):
    """
    Plot user planets on the P–ρ panel (including 3σ upper limits).

    Returns
    -------
    P_for_stats : np.ndarray or None
        Periods (days) of user planets included in the statistics
        (non–upper–limits only, within the Neptunian band).
    rho_for_stats : np.ndarray or None
        Densities (g cm^-3) of those user planets.
    sigma_rho_for_stats : np.ndarray or None
        1σ uncertainties on the densities (g cm^-3).
    labels_for_stats : list of str
        Names of the user planets used in the statistics.
    """
    (
        P_user,
        rho_user,
        sigma_rho_user,
        is_upper,
        labels,
        colors,
        dxs,
        dys,
        missing_names,
    ) = get_user_density_arrays()

    if missing_names:
        missing_sorted = ", ".join(sorted(set(missing_names)))
        print(
            "WARNING (user planets): not shown in the density panel "
            "due to missing mass/density information: "
            f"{missing_sorted}"
        )

    if P_user.size == 0:
        return None, None, None, []

    P_for_stats = []
    rho_for_stats = []
    sigma_rho_for_stats = []
    labels_for_stats = []
    handle_list = []

    for i in range(P_user.size):
        P      = P_user[i]
        rho    = rho_user[i]
        sig    = sigma_rho_user[i]
        up_lim = is_upper[i]
        name   = labels[i]
        col    = colors[i]
        dx     = dxs[i]
        dy     = dys[i]

        if not np.isfinite(P) or not np.isfinite(rho):
            continue

        if not up_lim:
            # Normal detection
            sc = ax.errorbar(
                P,
                rho,
                yerr=sig if (sig is not None and sig > 0) else None,
                fmt="h",
                mfc=col,
                mec="k",
                ms=user_marker_size_rho,
                ecolor="k",
                elinewidth=1.8,
                capsize=3,
                zorder=7,
                label=name if legend_or_box == "legend" else None,
            )
            handle_list.append(sc)
            P_for_stats.append(P)
            rho_for_stats.append(rho)
            sigma_rho_for_stats.append(sig if np.isfinite(sig) else 0.0)

        else:
            # 3σ upper limit: downward triangle with an "uplims" arrow.
            arrow_len = max(0.35 * rho, 0.2)
            ax.errorbar(
                P,
                rho,
                yerr=arrow_len,
                fmt="v",
                mfc=col,
                mec="k",
                ms=user_marker_size_rho_UL,
                ecolor="k",
                elinewidth=1.2,
                capsize=4,
                uplims=True,
                zorder=7,
                label=name if legend_or_box == "legend" else None,
            )

        if legend_or_box == "box":
            ax.text(
                P + dx,
                rho + dy,
                name,
                fontsize=10.5,
                va="bottom",
                ha="center",
                bbox=dict(
                    boxstyle="round",
                    facecolor=col,
                    alpha=0.85,
                    pad=0.4,
                ),
                c="white",
                weight="bold",
                zorder=10000,
            )

        # Para las estadísticas solo usamos detecciones (no upper limits)
        if not up_lim and np.isfinite(rho) and np.isfinite(P):
            labels_for_stats.append(name)

    if len(P_for_stats) == 0:
        return None, None, None, []

    return (
        np.asarray(P_for_stats),
        np.asarray(rho_for_stats),
        np.asarray(sigma_rho_for_stats),
        labels_for_stats,
    )

def cliffs_delta(x, y):
    """
    Compute Cliff's delta effect size between two 1D samples.
    """
    x = np.asarray(x)
    y = np.asarray(y)
    n1 = len(x)
    n2 = len(y)
    if n1 == 0 or n2 == 0:
        return np.nan
    greater, less = 0, 0
    for xi in x:
        greater += np.sum(xi > y)
        less    += np.sum(xi < y)
    return (greater - less) / (n1 * n2)


def _summary_median_16_84(arr):
    """
    Return median and 16th/84th percentile error bars.
    """
    arr = np.asarray(arr)
    arr = arr[np.isfinite(arr)]
    if arr.size == 0:
        return None
    p16, med, p84 = np.percentile(arr, [16, 50, 84])
    return med, (p84 - med), (med - p16)


# Aliases kept for clarity and backwards compatibility
summary_3dec = _summary_median_16_84
summary_4dec = _summary_median_16_84

In [ ]:
# Castro-González et al. (2024) Neptunian desert boundaries
# x_tw_* : log10(P / days), y_tw_* : log10(R / R_earth)

x_tw_low = np.linspace(-0.30, 0.47, 100)
x_tw_up  = np.linspace(0.120, 0.473, 100)

y_tw_up  = -0.43 * x_tw_up + 1.14
y_tw_low =  0.55 * x_tw_low + 0.36

# Mazeh et al. (2016) Neptunian desert boundaries in log10 P–R space
x            = np.linspace(-1.0, 1.163, 1000)
upper_radius = -0.33 * x + 1.17
lower_radius = 0.68 * x


def draw_mazeh_boundaries(
    ax,
    color="k",
    lw=2.5,
    linestyle=":",
    zorder=1e5,
    label=None,
):
    """Plot Mazeh et al. (2016) desert boundaries on a P–R axis."""
    ax.plot(
        x,
        upper_radius,
        c=color,
        lw=lw,
        linestyle=linestyle,
        zorder=zorder,
        label=label,
    )
    ax.plot(
        x,
        lower_radius,
        c=color,
        lw=lw,
        linestyle=linestyle,
        zorder=zorder,
    )

In [ ]:
def plot_neptunian_landscape_approx():
    """
    Plot the approximate Neptunian landscape in the P–R plane.
    """
    from matplotlib.lines import Line2D

    fig, ax = plt.subplots(figsize=(7.7, 7.3))

    # Local flags in case `show_boundaries` was edited after import
    boundaries_list = [s.strip() for s in show_boundaries.split(",") if s.strip()]
    show_desert_boundaries = "desert" in boundaries_list
    show_ridge_boundaries  = "ridge"  in boundaries_list

    # Catalogue background in P–R
    ax.scatter(
        x=np.log10(P_catalog),
        y=np.log10(R_catalog),
        ec="white",
        s=30,
        zorder=1,
        fc="grey",
        alpha=1.0,
        lw=0.7,
    )

    # Castro-González et al. (2024) desert boundaries
    if show_desert_boundaries:
        ax.vlines(
            x=[np.log10(2.95), -0.30, 0.12],
            ymin=[np.log10(4.1), -0.07, 1.09],
            ymax=[np.log10(8.4), 0.20, 1.20],
            colors=color_boundaries,
            linestyle=style_boundaries,
            lw=2.3,
            zorder=1e5,
        )
        ax.plot(
            x_tw_up,
            y_tw_up,
            c=color_boundaries,
            linestyle=style_boundaries,
            lw=2.3,
            zorder=1e4,
        )
        ax.plot(
            x_tw_low,
            y_tw_low,
            c=color_boundaries,
            linestyle=style_boundaries,
            lw=2.3,
            zorder=1e4,
        )

    # Optional ridge vertical segments (P = 2.95, 5.7 d)
    if show_ridge_boundaries:
        y_min_ridge = np.log10(4.5)
        y_max_ridge = np.log10(7.5)

        if not show_desert_boundaries:
            ax.vlines(
                x=[np.log10(2.95), np.log10(5.7)],
                ymin=[y_min_ridge, y_min_ridge],
                ymax=[y_max_ridge, y_max_ridge],
                colors=color_boundaries,
                linestyle=style_boundaries,
                lw=2.3,
                zorder=1e5,
            )
        else:
            ax.vlines(
                x=[np.log10(5.7)],
                ymin=[y_min_ridge],
                ymax=[y_max_ridge],
                colors=color_boundaries,
                linestyle=style_boundaries,
                lw=2.3,
                zorder=1e5,
            )

    # Optional Mazeh et al. (2016) boundaries
    if show_mazeh_boundaries:
        draw_mazeh_boundaries(ax)

    # KDE background levels
    if catalog == "NEA":
        levels = [0, 0.163, 0.165, 0.17, 0.18,
                  0.2, 0.25, 0.3, 0.4, 0.5, 0.6]
    elif catalog == "Exoplanet.eu":
        levels = [0, 0.19, 0.2, 0.25, 0.3,
                  0.4, 0.5, 0.6]
    else:
        levels = None

    # KDE background
    sns.kdeplot(
        x=logP_kde,
        y=logR_kde,
        cmap=color_map,
        fill=True,
        zorder=-1,
        alpha=1.0,
        bw_adjust=0.3,
        levels=levels,
        ax=ax,
    )

    # Region labels
    if show_label_desert:
        txt = ax.text(
            -0.25,
            0.74,
            "DESERT",
            fontname="Arial Black",
            zorder=100,
            fontsize=20,
            alpha=0.9,
            color=color_text,
            fontweight=600,
        )
        txt.set_path_effects(outlined_text_effect())

    if show_label_ridge:
        txt = ax.text(
            0.58,
            0.65,
            "RIDGE",
            rotation=90,
            fontname="Arial Black",
            zorder=100,
            fontsize=20,
            alpha=0.9,
            color=color_text,
            fontweight=600,
        )
        txt.set_path_effects(outlined_text_effect())

    if show_label_savanna:
        txt = ax.text(
            1.12,
            0.74,
            "SAVANNA",
            fontname="Arial Black",
            zorder=10e10,
            fontsize=20,
            alpha=0.9,
            color=color_text,
            fontweight=600,
        )
        txt.set_path_effects(outlined_text_effect())

    # User planets (P–R)
    planet_handles = plot_user_planets_PR(ax, legend_or_box)

    planet_legend = None
    if legend_or_box == "legend" and planet_handles:
        planet_labels = [h.get_label() for h in planet_handles]
        planet_legend = ax.legend(
            handles=planet_handles,
            labels=planet_labels,
            fontsize=12,
            loc="upper right",
            markerscale=0.8,
            frameon=True,
        )

    # Boundary legend: only when BOTH Castro-González and Mazeh
    # boundaries are plotted
    if show_desert_boundaries and show_mazeh_boundaries:
        boundary_handles = [
            Line2D([], [], color="k", linestyle=":", lw=2.5),
            Line2D([], [], color=color_boundaries,
                   linestyle=style_boundaries, lw=2.3),
        ]
        boundary_labels = [
            "Mazeh et al. (2016)",
            "Castro-González et al. (2024)",
        ]
        boundary_legend = ax.legend(
            boundary_handles,
            boundary_labels,
            loc="lower right",
            fontsize=11.5,
            frameon=True,
            handlelength=2.5,
            borderpad=0.8,
        )
        boundary_legend.get_frame().set_linewidth(0.8)

        # Re-add planet legend so that both legends are visible
        if planet_legend is not None:
            ax.add_artist(planet_legend)

    # Shared P–R axis formatting
    setup_PR_axes(ax)

    # Save figure with optional tag
    os.makedirs(output_dir, exist_ok=True)
    fig.savefig(
        os.path.join(
            output_dir,
            make_output_name("nep_lands.pdf"),
        ),
        bbox_inches="tight",
    )
    fig.savefig(
        os.path.join(
            output_dir,
            make_output_name("nep_lands.png"),
        ),
        bbox_inches="tight",
        dpi=400,
    )

    plt.show()

In [ ]:
def plot_neptunian_landscape_density():
    """
    Two–panel figure:
      - Left: P–R landscape.
      - Right: P–ρ density trend in the Neptunian range.
    """


    global max_rel_err_rho_pct, max_frac_err_R_pct, max_frac_err_M_pct

    # Defaults if the user forgot to define them
    if "max_rel_err_rho_pct" not in globals():
        max_rel_err_rho_pct = 0.0
    if "max_frac_err_R_pct" not in globals():
        max_frac_err_R_pct = 100.0
    if "max_frac_err_M_pct" not in globals():
        max_frac_err_M_pct = 100.0


    
    fig, (ax1, ax2) = plt.subplots(
        1, 2,
        figsize=(16.8, 7.3),
        gridspec_kw={"wspace": 0.18},
    )

    # ============================================================
    # LEFT PANEL — P–R
    # ============================================================

    ax1.scatter(
        x=np.log10(P_catalog),
        y=np.log10(R_catalog),
        ec="white",
        s=30,
        zorder=2,
        fc="grey",
        alpha=1.0,
        lw=0.7,
    )

    if catalog == "NEA":
        levels = [0, 0.163, 0.165, 0.17, 0.18,
                  0.2, 0.25, 0.3, 0.4, 0.5, 0.6]
    else:
        levels = [0, 0.19, 0.2, 0.25, 0.3,
                  0.4, 0.5, 0.6]

    sns.kdeplot(
        x=logP_kde,
        y=logR_kde,
        cmap=color_map,
        fill=True,
        zorder=-1,
        alpha=1.0,
        bw_adjust=0.3,
        levels=levels,
        ax=ax1,
    )

    if show_desert_boundaries:
        ax1.vlines(
            x=[np.log10(2.95), -0.30, 0.12],
            ymin=[np.log10(4.1), -0.07, 1.09],
            ymax=[np.log10(8.4), 0.20, 1.20],
            colors=color_boundaries,
            linestyle=style_boundaries,
            lw=2.3,
            zorder=10,
        )
        ax1.plot(
            x_tw_up,
            y_tw_up,
            c=color_boundaries,
            linestyle=style_boundaries,
            lw=2.3,
            zorder=10,
        )
        ax1.plot(
            x_tw_low,
            y_tw_low,
            c=color_boundaries,
            linestyle=style_boundaries,
            lw=2.3,
            zorder=10,
        )

    if show_ridge_boundaries and not show_desert_boundaries:
        y_min_ridge = np.log10(R_band_min)
        y_max_ridge = np.log10(R_band_max)
        ax1.vlines(
            x=[np.log10(2.95)],
            ymin=[y_min_ridge],
            ymax=[y_max_ridge],
            colors=color_boundaries,
            linestyle=style_boundaries,
            lw=2.3,
            zorder=10,
        )

    y_min_band = np.log10(R_band_min)
    y_max_band = np.log10(R_band_max)
    x_min      = -0.5
    x_max      = 2.2
    x_split    = np.log10(6.0)

    ax1.add_patch(
        patches.Rectangle(
            (x_min, y_min_band),
            width=x_split - x_min,
            height=y_max_band - y_min_band,
            facecolor=color_ridge_desert,
            edgecolor="none",
            alpha=alpha_band,
            zorder=1,
        )
    )
    ax1.add_patch(
        patches.Rectangle(
            (x_split, y_min_band),
            width=x_max - x_split,
            height=y_max_band - y_min_band,
            facecolor=color_savanna,
            edgecolor="none",
            alpha=alpha_band,
            zorder=1,
        )
    )

    if show_ridge_boundaries:
        ax1.vlines(
            x_split, y_min_band, y_max_band,
            colors=color_boundaries,
            linestyles=style_boundaries,
            lw=2.3,
            zorder=11,
        )

    if show_mazeh_boundaries:
        draw_mazeh_boundaries(ax1)

    if show_label_desert:
        txt = ax1.text(
            -0.25, 0.74,
            "DESERT",
            fontname="Arial Black",
            fontsize=19,
            alpha=0.9,
            color="white",
            fontweight=600,
            zorder=100,
        )
        txt.set_path_effects(
            [PathEffects.withStroke(linewidth=2.2, foreground="black")]
        )

    if show_label_ridge:
        txt = ax1.text(
            0.585, 0.673,
            "RIDGE",
            rotation=90,
            fontname="Arial Black",
            fontsize=19,
            alpha=0.9,
            color="white",
            fontweight=600,
            zorder=100,
        )
        txt.set_path_effects(
            [PathEffects.withStroke(linewidth=2.2, foreground="black")]
        )

    if show_label_savanna:
        txt = ax1.text(
            1.12, 0.74,
            "SAVANNA",
            fontname="Arial Black",
            fontsize=19,
            alpha=0.9,
            color="white",
            fontweight=600,
            zorder=10e10,
        )
        txt.set_path_effects(
            [PathEffects.withStroke(linewidth=2.2, foreground="black")]
        )

    # User planets (P–R panel)
    planet_handles = plot_user_planets_PR(ax1, legend_or_box)

    planet_legend = None
    if legend_or_box == "legend" and planet_handles:
        planet_labels = [h.get_label() for h in planet_handles]
        planet_legend = ax1.legend(
            handles=planet_handles,
            labels=planet_labels,
            fontsize=12,
            loc="upper right",
            markerscale=0.8,
            frameon=True,
        )

    # Boundary legend: only when BOTH Castro-González and Mazeh
    # boundaries are plotted
    if show_desert_boundaries and show_mazeh_boundaries:
        boundary_elems = [
            # First Mazeh (top entry)
            plt.Line2D(
                [0], [0],
                color="k",
                linestyle=":",
                lw=2.5,
                label="Mazeh et al. (2016)",
            ),
            # Then Castro-González (bottom entry)
            plt.Line2D(
                [0], [0],
                color=color_boundaries,
                linestyle=style_boundaries,
                lw=2.3,
                label="Castro-González et al. (2024)",
            ),
        ]
        boundary_legend = ax1.legend(
            handles=boundary_elems,
            loc="lower right",
            fontsize=11,
            frameon=True,
            handlelength=2.5,
            borderpad=0.8,
        )

        # Re-add planet legend so that both legends are visible
        if planet_legend is not None:
            ax1.add_artist(planet_legend)

    setup_PR_axes(ax1)

    # ============================================================
    # RIGHT PANEL — P–ρ
    # ============================================================

    mask_basic = (
        df_cat["pl_orbper"].notna() &
        df_cat["pl_rade"].notna() &
        df_cat["pl_bmasse"].notna()
    )

    mask_R = (
        (df_cat["pl_rade"] >= R_band_min) &
        (df_cat["pl_rade"] <= R_band_max)
    )

    R   = df_cat["pl_rade"].to_numpy()
    dR1 = df_cat["pl_radeerr1"].to_numpy()
    dR2 = df_cat["pl_radeerr2"].to_numpy()
    M   = df_cat["pl_bmasse"].to_numpy()
    dM1 = df_cat["pl_bmasseerr1"].to_numpy()
    dM2 = df_cat["pl_bmasseerr2"].to_numpy()

    sigma_R = 0.5 * (np.abs(dR1) + np.abs(dR2))
    sigma_M = 0.5 * (np.abs(dM1) + np.abs(dM2))

    frac_R = sigma_R / R
    frac_M = sigma_M / M

    with np.errstate(divide="ignore", invalid="ignore"):
        rho_all = rho_earth * (M / (R**3))

    sigma_rho_all = rho_all * np.sqrt(frac_M**2 + (3.0 * frac_R)**2)
    rho_rel_err   = sigma_rho_all / rho_all

    max_rel_err_rho = max_rel_err_rho_pct / 100.0
    max_frac_err_R  = max_frac_err_R_pct  / 100.0
    max_frac_err_M  = max_frac_err_M_pct  / 100.0

    if max_rel_err_rho_pct > 0.0:
        mask_prec = np.isfinite(rho_rel_err) & (rho_rel_err <= max_rel_err_rho)
    else:
        mask_prec = (frac_R < max_frac_err_R) & (frac_M < max_frac_err_M)

    mask = (
        mask_basic &
        mask_R &
        mask_prec &
        np.isfinite(rho_all) &
        np.isfinite(sigma_rho_all)
    )

    R_sel       = R[mask]
    M_sel       = M[mask]
    sigma_R_sel = sigma_R[mask]
    sigma_M_sel = sigma_M[mask]

    P_sel     = df_cat.loc[mask, "pl_orbper"].values
    rho_nom   = rho_all[mask]
    sigma_rho = sigma_rho_all[mask]

    ax2.axvspan(
        0.0, 6.0,
        ymin=0, ymax=1,
        facecolor=color_ridge_desert,
        alpha=alpha_band,
        edgecolor="none",
        zorder=0,
    )
    ax2.axvspan(
        6.0, density_xlim_max,
        ymin=0, ymax=1,
        facecolor=color_savanna,
        alpha=alpha_band,
        edgecolor="none",
        zorder=0,
    )
    ax2.axvline(
        6.0,
        color="k",
        linestyle="dashdot",
        lw=1.5,
        zorder=3,
    )

    ax2.errorbar(
        P_sel, rho_nom,
        yerr=sigma_rho,
        fmt="none",
        ecolor="k",
        elinewidth=1.8,
        capsize=0,
        alpha=0.95,
        zorder=2,
    )
    ax2.scatter(
        P_sel, rho_nom,
        s=63,
        facecolors="white",
        edgecolors="k",
        linewidths=1.1,
        zorder=4,
    )

    P_user_sel, rho_user_stats, sigma_rho_user_stats, labels_user_stats = \
    plot_user_planets_density(ax2, legend_or_box)

    if legend_or_box == "legend":
        planet_handles, planet_labels = ax2.get_legend_handles_labels()
    else:
        planet_handles, planet_labels = [], []

    ax2.axhline(
        1.0,
        color="k",
        linestyle="dotted",
        lw=1.2,
        zorder=3,
    )

    ax2.set_yticks([0.0, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0])
    ax2.set_yticklabels([0.0, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0], fontsize=13.5)

    ax2.set_xlim(0.0, density_xlim_max)
    ax2.set_ylim(0.0, density_ymax)
    ax2.set_xlabel("Orbital period (days)", fontsize=16)
    ax2.set_ylabel(r"Planet density (g cm$^{-3}$)", fontsize=16)

    ax2.tick_params(
        which="both",
        direction="in",
        length=6,
        width=1.3,
        top=True,
        right=True,
        color="k",
    )

    ax2.set_facecolor("white")

    legend_elems = [
        plt.Line2D(
            [0], [0],
            color=color_ridge_desert,
            lw=5,
            alpha=alpha_band,
            label="Desert + Ridge",
        ),
        plt.Line2D(
            [0], [0],
            color=color_savanna,
            lw=5,
            alpha=alpha_band,
            label="Savanna",
        ),
        plt.Line2D(
            [0], [0],
            color="k",
            linestyle="dotted",
            lw=1.5,
            label=r"$\rho = 1$",
        ),
    ]

    handles = list(legend_elems)
    labels  = [h.get_label() for h in legend_elems]

    # Planet symbols in the legend (hexagon vs triangle)
    if legend_or_box == "legend" and planet_labels:
        planet_colors = {pl.get("name"): pl.get("color") for pl in user_planets}
        planet_is_upper = {
            pl.get("name"): bool(
                pl.get("M_upper_3sigma", False)
                or pl.get("rho_upper_3sigma", False)
            )
            for pl in user_planets
        }

        for lab in planet_labels:
            col   = planet_colors.get(lab, "black")
            is_ul = planet_is_upper.get(lab, False)
            marker = "v" if is_ul else "h"

            clean = Line2D(
                [0], [0],
                marker=marker,
                markersize=user_marker_size_legend,
                markerfacecolor=col,
                markeredgecolor="k",
                linewidth=0,
                label=lab,
            )
            handles.append(clean)
            labels.append(lab)

    leg = ax2.legend(
        handles=handles,
        labels=labels,
        fontsize=11,
        loc="upper right",
        frameon=True,
        handlelength=2.5,
        borderpad=0.8,
    )
    leg.get_frame().set_linewidth(0.8)

    plt.show()

    # ============================================================
    # CONFIGURATION PRINT / STATS
    # ============================================================

    print("========== DENSITY TREND CONFIGURATION ==========")

    if max_rel_err_rho_pct > 0.0:
        print(
            "Max uncertainties: "
            f"σρ/ρ < {max_rel_err_rho_pct:.1f}% "
            f"(Catalog: {catalog})"
        )
    else:
        print(
            "Max uncertainties: "
            f"σR/R < {max_frac_err_R_pct:.1f}%, "
            f"σM/M < {max_frac_err_M_pct:.1f}% "
            f"(Catalog: {catalog})"
        )

    if include_user_planets_in_density_stats:
        if P_user_sel is not None and np.size(P_user_sel) > 0:
            names_for_stats = sorted(set(labels_user_stats))
            print(
                "Including user planets in density statistics: "
                + ", ".join(names_for_stats)
            )
        else:
            print(
                "Including user planets in density statistics: "
                "no user planets with measured density in the selected radius range."
            )
    else:
        print("User planets are NOT included in density statistics.")

    print(f"Max period in density panel: {density_xlim_max:.1f} days")
    print(f"Radius range: {R_band_min:.2f} R⊕ – {R_band_max:.2f} R⊕")
    print("")

    # ------------------------------------------------------------
    # SAMPLE SIZES: catalogue only, or catalogue + user planets
    # ------------------------------------------------------------
    desert_mask  = (P_sel >= 0.0) & (P_sel < 3.2)
    ridge_mask   = (P_sel >= 3.2) & (P_sel < 6.0)
    savanna_mask = (P_sel >= 6.0) & (P_sel <= density_xlim_max)

    # Catalogue-only counts
    N_desert_cat  = desert_mask.sum()
    N_ridge_cat   = ridge_mask.sum()
    N_savanna_cat = savanna_mask.sum()

    # Start from catalogue counts
    N_desert  = N_desert_cat
    N_ridge   = N_ridge_cat
    N_savanna = N_savanna_cat

    # Optionally add user planets to the sample sizes
    if (
        include_user_planets_in_density_stats
        and (P_user_sel is not None)
        and np.size(P_user_sel) > 0
    ):
        desert_mask_user  = (P_user_sel >= 0.0) & (P_user_sel < 3.2)
        ridge_mask_user   = (P_user_sel >= 3.2) & (P_user_sel < 6.0)
        savanna_mask_user = (P_user_sel >= 6.0) & (P_user_sel <= density_xlim_max)

        N_desert  += desert_mask_user.sum()
        N_ridge   += ridge_mask_user.sum()
        N_savanna += savanna_mask_user.sum()

        print("========== SAMPLE SIZES (catalogue + user planets) ==========")
        print(
            f"N_desert = {N_desert}  "
            f"(catalogue {N_desert_cat} + user {N_desert - N_desert_cat})"
        )
        print(
            f"N_ridge  = {N_ridge}  "
            f"(catalogue {N_ridge_cat} + user {N_ridge - N_ridge_cat})"
        )
        print(
            f"N_savanna = {N_savanna}  "
            f"(catalogue {N_savanna_cat} + user {N_savanna - N_savanna_cat})"
        )
    else:
        print("========== SAMPLE SIZES (catalogue only) ==========")
        print(
            f"N_desert = {N_desert_cat},  "
            f"N_ridge = {N_ridge_cat},  "
            f"N_savanna = {N_savanna_cat}"
        )
    print("")

    rho_nom_desert  = rho_nom[desert_mask]
    rho_nom_ridge   = rho_nom[ridge_mask]
    rho_nom_savanna = rho_nom[savanna_mask]

    N_mc = 5000
    rng  = np.random.default_rng(42)

    p_MW_dr   = []
    p_KS_dr   = []
    delta_dr  = []

    p_MW_ds   = []
    p_KS_ds   = []
    delta_ds  = []

    med_desert_list  = []
    med_ridge_list   = []
    med_savanna_list = []

    for _ in range(N_mc):
        M_mc = rng.normal(M_sel, sigma_M_sel)
        R_mc = rng.normal(R_sel, sigma_R_sel)

        M_mc = np.where(M_mc <= 0, M_sel, M_mc)
        R_mc = np.where(R_mc <= 0, R_sel, R_mc)

        rho_mc = rho_earth * (M_mc / (R_mc**3))

        rho_desert_mc  = rho_mc[desert_mask]
        rho_ridge_mc   = rho_mc[ridge_mask]
        rho_savanna_mc = rho_mc[savanna_mask]
        
        if (
            include_user_planets_in_density_stats
            and P_user_sel is not None
            and np.size(P_user_sel) > 0
        ):

            rho_user_mc = rng.normal(rho_user_stats, sigma_rho_user_stats)

            desert_mask_user  = (P_user_sel >= 0.0) & (P_user_sel < 3.2)
            ridge_mask_user   = (P_user_sel >= 3.2) & (P_user_sel < 6.0)
            savanna_mask_user = (P_user_sel >= 6.0) & (P_user_sel <= density_xlim_max)

            if rho_user_mc[desert_mask_user].size > 0:
                rho_desert_mc = np.concatenate(
                    [rho_desert_mc, rho_user_mc[desert_mask_user]]
                )
            if rho_user_mc[ridge_mask_user].size > 0:
                rho_ridge_mc = np.concatenate(
                    [rho_ridge_mc, rho_user_mc[ridge_mask_user]]
                )
            if rho_user_mc[savanna_mask_user].size > 0:
                rho_savanna_mc = np.concatenate(
                    [rho_savanna_mc, rho_user_mc[savanna_mask_user]]
                )

        if rho_desert_mc.size > 0:
            med_desert_list.append(np.median(rho_desert_mc))
        if rho_ridge_mc.size > 0:
            med_ridge_list.append(np.median(rho_ridge_mc))
        if rho_savanna_mc.size > 0:
            med_savanna_list.append(np.median(rho_savanna_mc))

        rho_desert_ridge_mc = np.concatenate([rho_desert_mc, rho_ridge_mc])

        if rho_desert_mc.size >= 3 and rho_ridge_mc.size >= 3:
            U_dr, p_dr = mannwhitneyu(
                rho_desert_mc,
                rho_ridge_mc,
                alternative="two-sided",
            )
            ks_dr = ks_2samp(rho_desert_mc, rho_ridge_mc)
            cd_dr = cliffs_delta(rho_desert_mc, rho_ridge_mc)
            p_MW_dr.append(p_dr)
            p_KS_dr.append(ks_dr.pvalue)
            delta_dr.append(cd_dr)

        if rho_desert_ridge_mc.size >= 3 and rho_savanna_mc.size >= 3:
            U_ds, p_ds = mannwhitneyu(
                rho_desert_ridge_mc,
                rho_savanna_mc,
                alternative="two-sided",
            )
            ks_ds = ks_2samp(rho_desert_ridge_mc, rho_savanna_mc)
            cd_ds = cliffs_delta(rho_desert_ridge_mc, rho_savanna_mc)
            p_MW_ds.append(p_ds)
            p_KS_ds.append(ks_ds.pvalue)
            delta_ds.append(cd_ds)

    print("=========== MEDIAN DENSITIES ===========")

    med_desert_arr  = np.array(med_desert_list)
    med_ridge_arr   = np.array(med_ridge_list)
    med_savanna_arr = np.array(med_savanna_list)

    if med_desert_arr.size > 0:
        m, up, lo = summary_3dec(med_desert_arr)
        print(f"Desert:   median = {m:.3f} (+{up:.3f} / -{lo:.3f}) g/cm^3")
    else:
        print("Desert:   median = NaN (no planets)")

    if med_ridge_arr.size > 0:
        m, up, lo = summary_3dec(med_ridge_arr)
        print(f"Ridge:    median = {m:.3f} (+{up:.3f} / -{lo:.3f}) g/cm^3")
    else:
        print("Ridge:    median = NaN (no planets)")

    if med_savanna_arr.size > 0:
        m, up, lo = summary_3dec(med_savanna_arr)
        print(f"Savanna:  median = {m:.3f} (+{up:.3f} / -{lo:.3f}) g/cm^3")
    else:
        print("Savanna:  median = NaN (no planets)")
    print("")

    print("====== (DESERT + RIDGE) vs SAVANNA ======")
    if len(p_MW_ds) > 0:
        m, up, lo = summary_4dec(np.array(p_MW_ds))
        print(f"Mann–Whitney p-value:       {m:.4f} (+{up:.4f} / -{lo:.4f})")
    if len(p_KS_ds) > 0:
        m, up, lo = summary_4dec(np.array(p_KS_ds))
        print(f"Kolmogorov–Smirnov p-value: {m:.4f} (+{up:.4f} / -{lo:.4f})")
    if len(delta_ds) > 0:
        m, up, lo = summary_4dec(np.array(delta_ds))
        print(f"Cliff's delta:              {m:.4f} (+{up:.4f} / -{lo:.4f})")
    print("")

    print("============ DESERT vs RIDGE ============")
    if len(p_MW_dr) > 0:
        m, up, lo = summary_4dec(np.array(p_MW_dr))
        print(f"Mann–Whitney p-value:       {m:.4f} (+{up:.4f} / -{lo:.4f})")
    if len(p_KS_dr) > 0:
        m, up, lo = summary_4dec(np.array(p_KS_dr))
        print(f"Kolmogorov–Smirnov p-value: {m:.4f} (+{up:.4f} / -{lo:.4f})")
    if len(delta_dr) > 0:
        m, up, lo = summary_4dec(np.array(delta_dr))
        print(f"Cliff's delta:              {m:.4f} (+{up:.4f} / -{lo:.4f})")
    print("")

    fig.savefig(
        os.path.join(
            output_dir,
            make_output_name("nep_dens.pdf"),
        ),
        bbox_inches="tight",
    )
    fig.savefig(
        os.path.join(
            output_dir,
            make_output_name("nep_dens.png"),
        ),
        dpi=400,
        bbox_inches="tight",
    )

In [ ]:
def plot_neptunian_landscape_exact():
    """
    Plot exact Neptunian desert boundaries on the Kepler DR25 sample.
    """
    fig, ax = plt.subplots(figsize=(7.7, 7.3))

    ax.set_axisbelow(False)
    for spine in ax.spines.values():
        spine.set_zorder(100)

    # DR25 weighted sample (log P, log R, errors, code)
    data = np.loadtxt(
        os.path.join(aux_dir, "DR25_data_weights.csv"),
        delimiter=",",
    )
    time     = data[:, 0]
    signal   = data[:, 1]
    err_down = data[:, 2]
    err_up   = data[:, 3]
    code     = data[:, 4]   # currently unused

    ax.scatter(
        time,
        signal,
        s=40,
        facecolors="white",
        edgecolors="black",
        linewidths=0.9,
        zorder=2,
    )

    yerr = np.vstack((err_down, err_up))
    ax.errorbar(
        time,
        signal,
        yerr=yerr,
        fmt="none",
        ecolor="black",
        elinewidth=0.8,
        alpha=0.5,
        zorder=1,
    )

    # Castro-González et al. (2024) desert boundaries
    if show_desert_boundaries:
        PX_MIN = -0.30
        PX_MAX =  0.60
        RY_MIN = -0.05
        RY_MAX =  1.26

        cont_data = np.loadtxt(
            os.path.join(aux_dir, "desert_boundaries.txt"),
            skiprows=1,
        )
        px_cont  = cont_data[:, 0]
        ry_cont  = cont_data[:, 1]
        lvl_cont = cont_data[:, 2]
        seg_cont = cont_data[:, 3]

        mask_cont = (
            (px_cont >= PX_MIN) & (px_cont <= PX_MAX) &
            (ry_cont >= RY_MIN) & (ry_cont <= RY_MAX)
        )
        px_cont  = px_cont[mask_cont]
        ry_cont  = ry_cont[mask_cont]
        lvl_cont = lvl_cont[mask_cont]
        seg_cont = seg_cont[mask_cont]

        first_contour = True

        for lev in np.unique(lvl_cont):
            mask_lev = (lvl_cont == lev)
            seg_ids  = np.unique(seg_cont[mask_lev])

            for sid in seg_ids:
                m = mask_lev & (seg_cont == sid)
                px_seg = px_cont[m]
                ry_seg = ry_cont[m]

                if len(px_seg) < 2:
                    continue

                # Clip to Mazeh upper boundary
                mazeh_upper_at_px = np.interp(px_seg, x, upper_radius)
                keep = (ry_seg <= mazeh_upper_at_px)
                px_seg = px_seg[keep]
                ry_seg = ry_seg[keep]

                if len(px_seg) < 2:
                    continue

                ax.plot(
                    px_seg,
                    ry_seg,
                    color="#1F3440",
                    linewidth=3.2,
                    alpha=0.95,
                    zorder=6,
                    label=(
                        "Castro-González et al. (2024)"
                        if first_contour else None
                    ),
                )
                first_contour = False

    # Mazeh et al. (2016) boundaries
    if show_mazeh_boundaries:
        draw_mazeh_boundaries(
            ax,
            color="k",
            label="Mazeh et al. (2016)",
        )

    # Neptunian ridge ellipse
    if show_ridge_boundaries:
        xmin_ridge = np.log10(2.95)
        xmax_ridge = np.log10(5.7)
        ymin_ridge = np.log10(4.0)
        ymax_ridge = np.log10(8.5)

        xc = 0.5 * (xmin_ridge + xmax_ridge)
        yc = 0.5 * (ymin_ridge + ymax_ridge)
        width  = xmax_ridge - xmin_ridge
        height = ymax_ridge - ymin_ridge
        xc_shifted = xc + 0.03

        ridge_ellipse = patches.Ellipse(
            (xc_shifted, yc),
            width=width,
            height=height,
            linewidth=2.0,
            edgecolor="#6A0DAD",
            facecolor="#6A0DAD33",
            zorder=3,
        )
        ax.add_patch(ridge_ellipse)

    # Region labels
    if show_label_desert:
        txt = ax.text(
            0.03,
            0.73,
            "NEPTUNIAN\nDESERT",
            ha="center",
            va="center",
            fontsize=20,
            fontweight="bold",
            color="white",
            linespacing=1.25,
            zorder=150,
        )
        txt.set_path_effects(
            [PathEffects.withStroke(linewidth=2.8, foreground="black")]
        )

    if show_label_ridge:
        txt = ax.text(
            0.6125,
            0.675,
            "RIDGE",
            rotation=90,
            fontname="Arial Black",
            zorder=150,
            fontsize=20,
            alpha=0.9,
            color="white",
            fontweight="bold",
        )
        txt.set_path_effects(
            [PathEffects.withStroke(linewidth=2.4, foreground="black")]
        )

    if show_label_savanna:
        txt = ax.text(
            0.93,
            0.74,
            "SAVANNA",
            fontname="Arial Black",
            zorder=10e10,
            fontsize=20,
            alpha=0.9,
            color="white",
            fontweight="bold",
        )
        txt.set_path_effects(
            [PathEffects.withStroke(linewidth=2.4, foreground="black")]
        )

    # User planets (P–R): get handles for the legend
    planet_handles = plot_user_planets_PR(ax, legend_or_box)

    planet_legend = None
    boundary_legend = None

    # ------------------------------------------------------------------
    # Legend for Castro-González & Mazeh boundaries
    # (only when BOTH sets of boundaries are shown)
    # ------------------------------------------------------------------
    handles, labels = ax.get_legend_handles_labels()
    desired = []
    if show_mazeh_boundaries and show_desert_boundaries:
        desired.extend([
            "Mazeh et al. (2016)",
            "Castro-González et al. (2024)",
        ])
    order = [labels.index(l) for l in desired if l in labels]

    if order:
        boundary_legend = ax.legend(
            handles=[handles[i] for i in order],
            labels=[labels[i] for i in order],
            loc="lower right",
            fontsize=12,
            frameon=True,
            markerscale=0.8,
        )
        boundary_legend.get_frame().set_linewidth(0.8)

    # ------------------------------------------------------------------
    # Planet legend:
    #  - if NO boundary legend: planets in lower-right corner
    #  - if boundary legend exists: planets just above it
    # ------------------------------------------------------------------
    if legend_or_box == "legend" and planet_handles:
        planet_labels = [h.get_label() for h in planet_handles]

        if boundary_legend is None:
            # No boundaries → planets in lower right
            planet_legend = ax.legend(
                handles=planet_handles,
                labels=planet_labels,
                fontsize=12,
                loc="lower right",
                markerscale=0.8,
                frameon=True,
            )
            planet_legend.get_frame().set_linewidth(0.8)
        else:
            # With boundaries → keep boundary legend in "lower right"
            # and place the planet legend just above it.
            fig.canvas.draw()
            bb = boundary_legend.get_window_extent().transformed(
                ax.transAxes.inverted()
            )
            y_top = bb.y1

            planet_legend = ax.legend(
                handles=planet_handles,
                labels=planet_labels,
                fontsize=12,
                loc="lower right",
                bbox_to_anchor=(1.0, y_top + 0.01),
                bbox_transform=ax.transAxes,
                markerscale=0.8,
                frameon=True,
            )
            planet_legend.get_frame().set_linewidth(0.8)

            ax.add_artist(boundary_legend)

    # Axes
    setup_PR_axes(ax)
    ax.set_xlim(-0.4, 1.50)
    ax.set_ylim(-0.045, 1.13)

    os.makedirs(output_dir, exist_ok=True)
    fig.savefig(
        os.path.join(
            output_dir,
            make_output_name("nep_des.pdf"),
        ),
        bbox_inches="tight",
    )
    fig.savefig(
        os.path.join(
            output_dir,
            make_output_name("nep_des.png"),
        ),
        dpi=400,
        bbox_inches="tight",
    )

    plt.show()

## 1. The exo-Neptunian landscape (desert, ridge, and savanna)
#### Period–radius diagram with the simple, ready-to-use approximations to the desert boundaries

In [ ]:
plot_neptunian_landscape_approx()

## 2. The super-Neptune / sub-Jovian density trend
#### Exo-Neptunian landscape (left panel) and desert+ridge vs. savanna density trend (right panel)

In [ ]:
plot_neptunian_landscape_density()

## 3. The Neptunian desert
#### Period–radius diagram with the exact (KDE-derived) desert boundaries

In [ ]:
plot_neptunian_landscape_exact()